In [2]:
import numpy
import matplotlib.pyplot as plt
import pandas
from astropy.io import fits
from glob import glob
from scipy.optimize import curve_fit

In [3]:
def get_flats(path):
    """
    Extracts flats from fits files
    
    Parameters
    ----------
    path: str
        path of all flat frames from rho
    Returns
    -------
    X_flats: np.ndarray
        3d array of flat frames in each filter (R, V, B, I)
    """

    flats_filenames = glob(path)
    flats = []
    flats_filters = []

    for i in range(len(flats_filenames[1:])):
        flats.append(fits.getdata(flats_filenames[i]))
        hdul = fits.open(flats_filenames[i])
        flats_filters.append(hdul[0].header['FILTER'])
        hdul.close()

    flats_filters = np.asarray(flats_filters)
    flats = np.asarray(flats)
    flats = flats.astype(float)

    R_flats = flats[np.where(flats_filters == 'R')]
    V_flats = flats[np.where(flats_filters == 'V')]
    B_flats = flats[np.where(flats_filters == 'B')]
    I_flats = flats[np.where(flats_filters == 'I')]
    
    return R_flats, V_flats, B_flats, I_flats

In [ ]:
_, V_flats, _, _ = get_flats('2020-10-17/h_persei_flats*')

In [4]:
def seq(path, expt):
    """
    Extracts light frames & darks from fits files
    
    Parameters
    ----------
    path: str
        Path of all flat frames from rho
    expt: float
        Exposure time of darks
    Returns
    -------
    X_seq: np.ndarray
        3d array of light frames in each filter (R, V, B, I)
    seq_darks: np.ndarra
        3d array of dark frames
    """


    seq_filenames = np.sort(glob(path))
    seq_light = []
    seq_darks = []
    seq_filters = []

    for i in range(len(seq_filenames[1:])):

        hdul = fits.open(seq_filenames[i])
        if hdul[0].header['FRAME'] == 'Light':
            seq_filters.append(hdul[0].header['FILTER'])
            seq_light.append(fits.getdata(seq_filenames[i]))
        else:
            seq_darks.append(fits.getdata(seq_filenames[i]))
        hdul.close()

    seq_filters = np.asarray(seq_filters)
    seq_light = np.asarray(seq_light)
    seq_light = seq_light.astype(float)


    R_seq = seq_light[np.where(seq_filters == 'R')]
    V_seq = seq_light[np.where(seq_filters == 'V')]
    B_seq = seq_light[np.where(seq_filters == 'B')]
    I_seq = seq_light[np.where(seq_filters == 'I')]
    
    return R_seq, V_seq, B_seq, I_seq, seq_darks

In [ ]:
R_seq, V_seq, B_seq, I_seq, seq_dark = seq('2020-10-17/h_persei_seq*', 5.0)

### Step 2: Master Dark:

In [ ]:
master_V_dark = np.median(seq_dark, axis=0)

### Step 3: Master Flat:

In [ ]:
comb_V_flat = np.median(V_flats,axis=0) - master_V_dark
div = master_V_dark/np.median(comb_V_flat)
master_V_flat = comb_V_flat - div

### Step 4, 5: Subtract dark and divide by flat for each science frame

In [ ]:
V_proc = []

for i in range(len(V_seq)):
    V_proc_temp = V_seq[i] - master_V_dark
    V_proc.append(V_proc_temp/master_V_flat)

### Step 6: Shifting frames with np.roll

In [ ]:
shifts = [[0,4], [0,7], [0,7], [-1,8], [-2,7], [-3,5], [-6,3], [-8,2], [-10,-1]]

In [5]:
def shiftroll(shifts, frames):
    """
    Shifts off-center frames, median-combines, and flips along y-axis
    
    Parameters
    ----------
    shifts: list
        list of [x,y] shifts
    frames: np.ndarray
        3d array of processed science images
    Returns
    -------
    rolledframes: np.ndarray
        Final median-combined image
    """
    rolledframes = []
    rolledframes.append(V_seq[0])
    for i in range(1, len(frames)):
        rolledframes.append(np.roll(frames[i], shifts[i-1], axis=(0,1)))
    
    rolledframes = np.asarray(rolledframes)
    rolledframes = np.median(rolledframes, axis=0)
    
    #flip to match DS9 orientation
    rolledframes = np.flip(rolledframes, 0)
    
    return rolledframes

In [ ]:
V = shiftroll(shifts, V_proc)

## Aperture Photometry

In [ ]:
from astropy.modeling import models, fitting

In [6]:
def cutout(image, bb, center):
    """
    Cuts out smaller frame around star
    
    Parameters
    ----------
    image: np.ndarray
        Whole image
    bb: int
        Cut out a box of side size 2*bb
    center: [x,y]
        Center of star in whole image [x,y]
    
    Returns
    -------
    xp, yp: ints
        Grids of box size
    box: np.ndarray
        Cutout image
    """
    xc = int(center[0])
    yc = int(center[1])
    box = image[yc-bb:yc+bb, xc-bb:xc+bb]
    xp, yp = box.shape
    return xp, yp, box

def gaussian(yp, xp, box):
    """ Fits a 2d Gaussian to star
    """
    y, x, = np.mgrid[:yp, :xp]
    f_init = models.Gaussian2D()
    fit_f = fitting.LevMarLSQFitter()
    fV = fit_f(f_init, x, y, box)
    
    return fV, x, y

def sigma_avg(x,y):
    """ Averages sigmas
    """
    return np.mean([float(x), float(y)])

def px_to_arcsec(fwhm):
    return fwhm*0.605

In [7]:
def psf(image, targ_loc, boxsize, expt, aper, aper_inside, aper_outside, plot='star'):
    """
    Star psf
    
    Parameters
    ----------
    image: np.ndarray
        Entire image
    targ_loc: [x,y]
        Coordinates of target
    boxsize: int
        1/2 box side length for image cutout around target star
    expt: float
        Exposure time
    aper: float
        Aperture size (units of FWHM)
    aper_inside: float
        Inner aperture size (units of FWHM)
    aper_outside: float
        Outside aperture size (units of FWHM)
    plot: 'star', 'gaussian', 'cutout'
        Plot keyword
    
    Returns
    -------
    F_targ: float
        Flux in PSF of target
    F_targ_persec: float
        Flux in PSF of target per second of exposure
    """
    F_sky_pix = []
    
    targ_loc[1] = 1266-targ_loc[1]
    
    xp, yp, current_image = cutout(image, boxsize, targ_loc)
    
    fV, x, y = gaussian(xp, yp, current_image)
    
    #plt.xlabel('x pixels')
    #plt.ylabel('y pixels')
    
    if plot=='star':
        plt.imshow(current_image)
        plt.show()
    elif plot =='gaussian':
        plt.imshow(fV(x,y))
        plt.show()
    
    sigma = sigma_avg(fV.x_stddev[0],fV.y_stddev[0])
    #xbar = fV.x_mean[0]
    fwhm = 2.355*float(sigma)
    fit = fV(x, y)
    
    center = np.where(fit == np.max(fit))
 
    circ_image = np.zeros(current_image.shape)
    N_targ = 0
    for i in range(len(current_image)):
        for j in range(len(current_image[0])):
            dist = np.sqrt((center[0]-i)**2+(center[1]-j)**2)
            if dist >= aper_inside*fwhm and dist <= aper_outside*fwhm:
                F_sky_pix.append(current_image[i][j])  
            if (dist > aper*fwhm):
                circ_image[i][j] = 0.0
            else:
                circ_image[i][j] = current_image[i][j]
                N_targ+=1

    F_sky_pix_med = np.median(F_sky_pix)
    F_targ_raw = 0
    F_targ_raw = np.sum(circ_image)
      
    if plot=='cutout':
        plt.imshow(circ_image)
        plt.show()
    
    F_targ = F_targ_raw - N_targ*F_sky_pix_med
    F_targ_persec = F_targ/expt
                        
    return F_targ_persec

In [8]:
tnames = ['2-3694-3626-1', '2-3694-2565-1', '2-3694-2943-1', '2-3694-1712-1', '2-3694-3750-1', '2-3694-3451-1', '2-3694-1822-1', '2-3694-1719-1',  '2-3694-1853-1', '2-3694-1363-1']
targlocs = [[440, 134], [382,1013], [529, 634], [892, 1139], [194, 272], [329,956], [1150, 146], [1032, 812], [1463, 134], [1485, 1077]]
boxsizes = [25, 35, 45, 30, 36, 40, 40, 27, 20, 56] 

In [9]:
def get_flux_per_s(image, targlocs, boxsizes, aper, aper_in, aper_out, plot=None):
    
    tnames = ['2-3694-3626-1', '2-3694-2565-1', '2-3694-2943-1', '2-3694-1712-1', '2-3694-3750-1', '2-3694-3451-1', '2-3694-1822-1', '2-3694-1719-1',  '2-3694-1853-1', '2-3694-1363-1']
    targlocs = [[440, 134], [382,1013], [529, 634], [892, 1139], [194, 272], [329,956], [1150, 146], [1032, 812], [1463, 134], [1485, 1077]]
    boxsizes = [25, 35, 45, 30, 36, 40, 40, 27, 20, 56]
    
    Flux_per_s = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    
    for i in range(len(targlocs)):
        F_targ_pers = psf(image, targlocs[i], boxsizes[i], 5.0, aper, aper_in, aper_out, plot)
        Flux_per_s[i] = F_targ_pers
    return Flux_per_s

In [10]:
Flux_per_s = get_flux_per_s(V, targlocs, boxsizes, 1.3, 2., 3., plot=None)

NameError: name 'V' is not defined